In [1]:

import os
import xlrd

from gurobipy import *


In [2]:
# ------------------------ input path and output path --------------------------------
path_inputData = "sample_data4.xls"
path_ModelSave = "model.mps"

In [3]:
diet = xlrd.open_workbook(path_inputData)

sh = diet.sheet_by_name("categories")
categories = []
minNutrition = {}
maxNutrition = {}
i = 1
while True:
    try:
        c = sh.cell_value(i, 0)
        categories.append(c)
        minNutrition[c] = sh.cell_value(i,1)
        maxNutrition[c] = sh.cell_value(i,2)
        i = i + 1
    except IndexError:
        break

sh = diet.sheet_by_name("foods")
foods = []
cost = {}
i = 1
while True:
    try:
        f = sh.cell_value(i, 0)
        foods.append(f)
        cost[f] = sh.cell_value(i,1)
        print(cost[f])
        i = i + 1
    except IndexError:
        break

sh = diet.sheet_by_name("nutrition_quantities")
nutritionValues = {}
for i in range(len(foods)):
    for j in range(len(categories)):
        nutritionValues[sh.cell_value(4*i+j+1,0),sh.cell_value(4*i+j+1,1)] = sh.cell_value(4*i+j+1,2)

m = Model("diet")


2.49
2.49
1.5
1.89
2.09
2.89
0.89
1.59
1.99
Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-13


In [4]:
def solve(categories, minNutrition, maxNutrition, foods, cost,
          nutritionValues):
    # Model


    # Create decision variables for the nutrition information,
    # which we limit via bounds
    nutrition = {}
    for c in categories:
        nutrition[c] = m.addVar(lb=minNutrition[c], ub=maxNutrition[c], name=c)
        print(nutrition[c])
    print("\n")

    # Create decision variables for the foods to buy
    buy = {}
    for f in foods:
        buy[f] = m.addVar(obj=cost[f], name=f)
        

    # The objective is to minimize the costs
    m.modelSense = GRB.MINIMIZE

    # Update model to integrate new variables
    m.update()

    # Nutrition constraints
    for c in categories:
        m.addConstr(
          quicksum(nutritionValues[f,c] * buy[f] for f in foods) ==
                    nutrition[c], c)
    
    def printSolution():
        if m.status == GRB.status.OPTIMAL:
            print ('\nCost:', m.objVal)
            print ('\nBuy:')
            for f in foods:
                if buy[f].x > 0.0001:
                    print (f, buy[f].x)
            print ('\nNutrition:')
            for c in categories:
                print (c, nutrition[c].x)
        else:
            print ('No solution')

    # Solve
    m.optimize()
    printSolution()


In [5]:
solve(categories, minNutrition, maxNutrition,
                foods, cost, nutritionValues)


<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 13 columns and 40 nonzeros
Model fingerprint: 0x7c5af4b8
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [9e-01, 2e+03]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 4 rows, 11 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.636375e+01   0.000000e+00      0s
       2    8.0336241e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.033624088e+00

Cost: 8.03362408759124

Buy:
hot dog 0.5333941605839416
ice cream 4.54939172749391

In [6]:

m.write(path_ModelSave)